In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [4]:
dict_margin = {'AD': 3000, 'BP': 3000, 'C': 2000, 'CD': 3000, 'CL': 18000, 'EC': 3800, 'ES': 13000, 'FDAX': 40000, 'GC': 15000, 'HG': 10000, 'NG': 5000, 'NQ': 23000,
               'RTY': 8000, 'S': 4000, 'VX': 26000, 'VXX': 900, 'YM': 15000}

In [5]:
def compute_np_dd(daily_profit: np.array) -> tuple[np.array, np.array, np.array]:
    '''
    Function to compute NP, equity peak and DD as functions of time.

    Args:
        daily_profit: Array containing the daily profit.
    
    Returns:
        cum_profit: Array containing cumulated profit over time.
        max_equity: Array containing equity peak over time.
        dd: Array containing drawdown over time.
    '''
    cum_profit = daily_profit.cumsum()
    max_equity = pd.Series(cum_profit).cummax().values
    dd = cum_profit - max_equity
    #
    return cum_profit, max_equity, dd

def correlation_classic(data_1: np.array, data_2: np.array) -> float:
    '''
    Function to compute the correlation between two strategies as lag-0 cross-correlation.

    Args:
        data_1: Array containing two columns: the first one are the dates, and the second one the P/L.
        data_2: Array containing two columns: the first one are the dates, and the second one the P/L.

    Returns:
        corr: Correlation coefficient.
    '''
    # get common dates and slice data
    _, idx_1, idx_2 = np.intersect1d(data_1['date'].values, data_2['date'].values, return_indices = True)
    data_1 = data_1.loc[idx_1, 'daily_profit'].astype(float)
    data_2 = data_2.loc[idx_2, 'daily_profit'].astype(float)
    n = data_1.shape[0]
    # mean values and standard deviations
    mu_1 = data_1.mean()
    mu_2 = data_2.mean()
    sigma_1 = data_1.std()
    sigma_2 = data_2.std()
    # compute correlation
    corr = 1/(n*sigma_1*sigma_2)*np.sum((data_1 - mu_1)*(data_2 - mu_2))
    #
    return corr

def _read_strats():
    '''
    Function to import strategies results.
    '''
    dict_results = {}
    for strat in list_strat:#self.dict_strat.keys():
        instrument = strat.split('_')[0]
        # read data
        with open(f'../reports/{strat}') as f:
            data = f.readlines()
        data = np.array([i.split(' ') for i in data])
        # get strategy parameters
        dates = pd.to_datetime(data[:, 0], format = '%d/%m/%Y')
        daily_profit = data[:, 1].astype(float)
        curr_contract = data[:, 2].astype(float)*dict_strat[strat]
        n_trades = data[:, 5].astype(int)
        # build dataframe
        df = pd.DataFrame({'date': dates, 'daily_profit': daily_profit, 'curr_contract': curr_contract, 'n_trades': n_trades})
        df['margin'] = dict_margin[instrument]*df['curr_contract'].abs()
        dict_results[strat] = df
    return dict_results

def portfolio_performance(dict_strat_vol: dict, date_start: str = None, date_end: str = None) -> pd.DataFrame:
        '''
        Function to compute the performance of a portfolio.
        
        Args:
            dict_strat_vol: Dictionary containing the number of contracts used for each strategy in the portfolio.
            date_start: First date to be considered.
            date_end: Last date to be considered.

        Returns:
            df_portfolio: Dataframe containing the portfolio performance.
        '''
        #
        df_portfolio = {instr: [] for instr in np.unique([strat.split('_')[0] for strat in dict_strat_vol.keys()])}
        for i in range(len(dict_strat_vol)):
            # get strategy features
            strategy = list(dict_strat_vol.keys())[i]
            instrument = strategy.split('_')[0]
            n_contracts = dict_strat_vol[strategy]
            #
            df_temp = dict_results[strategy]
            # keep relevant dates
            df_temp = df_temp[(df_temp['date'] >= date_start) & (df_temp['date'] < date_end)].reset_index(drop = True)
            # adjust profit and margins by number of contracts
            df_temp['daily_profit'] *= n_contracts
            df_temp['margin'] *= n_contracts*np.sign(df_temp['curr_contract'])
            # combine portfolio strategies
            df_temp.index = df_temp['date']
            df_portfolio[instrument].append(df_temp)
        df_portfolio = {instr: performance for instr, performance in df_portfolio.items() if len(performance) > 0}
        # combine performances for each instrument
        for instr in df_portfolio.keys():
            df_portfolio[instr] = pd.concat(df_portfolio[instr], axis = 1)
            if type(df_portfolio[instr]['daily_profit']) == pd.DataFrame:
                df_portfolio[instr]['daily_profit'] = df_portfolio[instr]['daily_profit'].sum(axis = 1)
            if type(df_portfolio[instr]['margin']) == pd.DataFrame:
                df_portfolio[instr]['margin'] = df_portfolio[instr]['margin'].sum(axis = 1)
            df_portfolio[instr] = df_portfolio[instr].loc[:, ~df_portfolio[instr].columns.duplicated()][['date', 'daily_profit', 'margin']]
            df_portfolio[instr]['margin'] = abs(df_portfolio[instr]['margin'])
        # compute aggregated performance
        df_portfolio = [value for value in df_portfolio.values()]
        df_portfolio = pd.concat(df_portfolio, axis = 1)
        if type(df_portfolio['daily_profit']) == pd.DataFrame:
            df_portfolio['daily_profit'] = df_portfolio['daily_profit'].sum(axis = 1)
        if type(df_portfolio['margin']) == pd.DataFrame:
            df_portfolio['margin'] = df_portfolio['margin'].sum(axis = 1)
        df_portfolio = df_portfolio.loc[:, ~df_portfolio.columns.duplicated()][['date', 'daily_profit', 'margin']]
        # add portfolio cumulative statistics
        df_portfolio['cum_profit'], df_portfolio['max_equity'], df_portfolio['dd'] = compute_np_dd(df_portfolio['daily_profit'])
        #
        return df_portfolio.reset_index(drop = True).sort_values(by = 'date')

In [7]:
list_strat = os.listdir('../reports/')
list_strat = [i for i in list_strat if 'SLA' not in i and 'GOOG' not in i]

In [9]:
dict_strat = dict(zip(list_strat, [1]*len(list_strat)))

In [10]:
dict_results = _read_strats()

In [16]:
dict_corr = {}
for i in range(len(dict_strat)):
    for j in range(i, len(dict_strat)):
        strat_1 = list(dict_strat.keys())[i]
        strat_2 = list(dict_strat.keys())[j]
        data_1, data_2 = dict_results[strat_1], dict_results[strat_2]
        dict_corr[('_'.join(strat_1.split('_')[1:]),
                   '_'.join(strat_2.split('_')[1:]))] = round(correlation_classic(data_1[['date', 'daily_profit']],
                                                                                  data_2[['date', 'daily_profit']]), 3)

In [88]:
df = portfolio_performance(dict_strat_vol = dict_strat, date_start = '2012-01-01', date_end = '2032-12-31')

In [114]:
figure = make_subplots(rows = 2, cols = 1, shared_xaxes = True, row_heights = [0.67, 0.33], vertical_spacing = 0)
figure.update_layout(go.Layout(margin = dict(l = 20, r = 20, t = 20, b = 20), template = 'simple_white', showlegend = False,
                            xaxis1 = {'showgrid': True, 'showline': True, 'mirror': True, 'titlefont': {'size': 20}, 'tickfont': {'size': 16},
                                        'tickangle': -90},
                            yaxis1 = {'showgrid': True, 'showline': True, 'mirror': True, 'titlefont': {'size': 20}, 'tickfont': {'size': 16},
                                        'tickformat': f'.{2}f', 'title': 'NP [$]'},
                            xaxis2 = {'showgrid': True, 'showline': True, 'mirror': True, 'titlefont': {'size': 20}, 'tickfont': {'size': 16},
                                        'tickangle': -90},
                            yaxis2 = {'showgrid': True, 'showline': True, 'mirror': True, 'titlefont': {'size': 20}, 'tickfont': {'size': 16},
                                        'tickformat': f'.{2}f', 'title': 'DD [$]'},
                            font = {'size': 28}, autosize = False, width = 900, height = 500, hovermode = 'closest'))
#
figure.add_trace(go.Scatter(x = df['date'].values, y = df['cum_profit'].values, mode = 'lines', line_color = 'blue'), row = 1, col = 1)
figure.add_trace(go.Scatter(x = df['date'].values, y = df['dd'].values, mode = 'lines', line_color = 'blue'), row = 2, col = 1)

In [358]:
dict_results.keys()

dict_keys(['AD__Sign_AD_Ranger_Trend_m60.txt', 'BP__Sign_BP_BiasIntraweek_m15.txt', 'BP__Sign_BP_Swing_Trend_m15.txt', 'CD__Sign_CD_Donchian_Counter_m15_m1440.txt'])

In [363]:
dict_corr = {}
for strat_1 in dict_strat.keys():
    for strat_2 in dict_strat.keys():
        data_1, data_2 = dict_results[strat_1], dict_results[strat_2]
        dict_corr[('_'.join(strat_1.split('_')[1:]),
                   '_'.join(strat_2.split('_')[1:]))] = round(correlation_classic(data_1[['date', 'daily_profit']],
                                                                                  data_2[['date', 'daily_profit']]), 3)
#
df_corr = pd.DataFrame(np.array(list(dict_corr.values())).reshape(int(np.sqrt(len(dict_corr))), -1))
df_corr.columns = ['_'.join(i.split('_')[1:]) for i in dict_strat.keys()]
df_corr.index = ['_'.join(i.split('_')[1:]) for i in dict_strat.keys()]
#
figure = go.Figure()
figure.update_layout(go.Layout(xaxis = {'showgrid': True, 'showline': True, 'mirror': True, 'titlefont': {'size': 20}, 'tickfont': {'size': 12}, 'tickangle': -40}))
figure.add_trace(go.Heatmap(z = df_corr.values[::-1, :], x = df_corr.columns, y = df_corr.index[::-1],
                            colorscale = 'Spectral_r', text = df_corr.values[::-1, :], texttemplate="%{text}", zmin = -1,
                            zmax = 1))

In [153]:
# figure = make_subplots(rows = 2, cols = 1, shared_xaxes = True, row_heights = [0.67, 0.33], vertical_spacing = 0)
# figure.update_layout(go.Layout(margin = dict(l = 20, r = 20, t = 20, b = 20), template = 'simple_white', showlegend = False,
#                             xaxis1 = {'showgrid': True, 'showline': True, 'mirror': True, 'titlefont': {'size': 20}, 'tickfont': {'size': 16},
#                                         'tickangle': -90},
#                             yaxis1 = {'showgrid': True, 'showline': True, 'mirror': True, 'titlefont': {'size': 20}, 'tickfont': {'size': 16},
#                                         'tickformat': f'.{2}f', 'title': 'NP [$]'},
#                             xaxis2 = {'showgrid': True, 'showline': True, 'mirror': True, 'titlefont': {'size': 20}, 'tickfont': {'size': 16},
#                                         'tickangle': -90},
#                             yaxis2 = {'showgrid': True, 'showline': True, 'mirror': True, 'titlefont': {'size': 20}, 'tickfont': {'size': 16},
#                                         'tickformat': f'.{2}f', 'title': 'DD [$]'},
#                             font = {'size': 28}, autosize = False, width = 900, height = 500, hovermode = 'closest'))
# #
# for i in range(100):
#     df_temp = df.copy()
#     shuffle_idx = np.random.choice(range(df_temp.shape[0]), df_temp.shape[0], replace = False)
#     df_temp['daily_profit'] = df_temp.loc[shuffle_idx, 'daily_profit'].values
#     df_temp['cum_profit'], _, df_temp['dd'] = compute_np_dd(df_temp['daily_profit'])
#     figure.add_trace(go.Scatter(x = df_temp['date'].values, y = df_temp['cum_profit'].values, mode = 'lines', line_color = 'gray', opacity = 0.5), row = 1, col = 1)
#     figure.add_trace(go.Scatter(x = df_temp['date'].values, y = df_temp['dd'].values, mode = 'lines', line_color = 'gray', opacity = 0.5), row = 2, col = 1)
# #
# figure.add_trace(go.Scatter(x = df['date'].values, y = df['cum_profit'].values, mode = 'lines', line_color = 'blue'), row = 1, col = 1)
# figure.add_trace(go.Scatter(x = df['date'].values, y = df['dd'].values, mode = 'lines', line_color = 'blue'), row = 2, col = 1)
# figure

In [170]:
q_3_iqr

126000.0

In [271]:
capital = 100000
prob_ruin = {}
for max_dd_accepted_perc in [10, 20, 30, 50, 100]:
    counter_tot, counter_ruin = {0: 0, 20: 0, 40: 0, 100: 0, 200: 0, 400: 0}, {0: 0, 20: 0, 40: 0, 100: 0, 200: 0, 400: 0}
    profit_mean = np.mean(df[df['daily_profit'] != 0]['daily_profit'].values)
    profit_std = np.std(df[df['daily_profit'] != 0]['daily_profit'].values)
    for i in range(1000):
        for n_days in counter_tot.keys():
            history_profit = capital + np.cumsum(np.random.normal(loc = profit_mean, scale = profit_std, size = n_days))
            if np.sum(history_profit <= capital*(1 - max_dd_accepted_perc/100)) > 0:
                counter_ruin[n_days] += 1
            counter_tot[n_days] += 1
    prob_ruin[max_dd_accepted_perc] = {n_days: round(counter_ruin[n_days]/counter_tot[n_days], 3) for n_days in counter_tot.keys()}

In [269]:
figure = go.Figure()
figure.update_layout(go.Layout(margin = dict(l = 20, r = 20, t = 20, b = 20), template = 'simple_white', showlegend = True,
                               legend = {'font': {'size': 13}, 'x': 0.},
                               xaxis = {'showgrid': True, 'showline': True, 'mirror': True, 'titlefont': {'size': 20}, 'tickfont': {'size': 16},
                                         'tickformat': f'.{0}f', 'title': 'Number of days'},
                               yaxis = {'showgrid': True, 'showline': True, 'mirror': True, 'titlefont': {'size': 20}, 'tickfont': {'size': 16},
                                         'tickformat': f'.{0}%', 'title': 'Probability of ruin'},
                               font = {'size': 28}, autosize = False, width = 900, height = 500, hovermode = 'closest'))
for max_dd_accepted_perc in prob_ruin.keys():
    figure.add_trace(go.Scatter(x = list(prob_ruin[max_dd_accepted_perc].keys()),
                                y = list(prob_ruin[max_dd_accepted_perc].values()), name = f'Max DD = {max_dd_accepted_perc}%'))
figure

In [272]:
figure = go.Figure()
figure.update_layout(go.Layout(margin = dict(l = 20, r = 20, t = 20, b = 20), template = 'simple_white', showlegend = True,
                               legend = {'font': {'size': 13}, 'x': 0.},
                               xaxis = {'showgrid': True, 'showline': True, 'mirror': True, 'titlefont': {'size': 20}, 'tickfont': {'size': 16},
                                         'tickformat': f'.{0}f', 'title': 'Number of days'},
                               yaxis = {'showgrid': True, 'showline': True, 'mirror': True, 'titlefont': {'size': 20}, 'tickfont': {'size': 16},
                                         'tickformat': f'.{0}%', 'title': 'Probability of ruin'},
                               font = {'size': 28}, autosize = False, width = 900, height = 500, hovermode = 'closest'))
for max_dd_accepted_perc in prob_ruin.keys():
    figure.add_trace(go.Scatter(x = list(prob_ruin[max_dd_accepted_perc].keys()),
                                y = list(prob_ruin[max_dd_accepted_perc].values()), name = f'Max DD = {max_dd_accepted_perc}%'))
figure

In [206]:
import pickle
aa = pickle.load(open('../aa.pickle', 'rb'))
list_first_dates = np.unique([df_temp['date'].min() for list_df in aa.values() for df_temp in list_df])
if list_first_dates.shape[0] > 1:
    print(f'The backtest starts on {np.max(list_first_dates).strftime("%Y-%m-%d")} because that is the first available date of at least one strategy.')
    for instr in aa.keys():
        list_df = aa[instr]
        for i in range(len(list_df)):
            df_temp = list_df[i]
            df_temp = df_temp[df_temp['date'] >= np.max(list_first_dates).strftime("%Y-%m-%d")].reset_index(drop = True)
            df_temp['n_trades'] -= df_temp['n_trades'].min()
            list_df[i] = df_temp

The backtest starts on 2012-01-01 because that is the first available date of at least one strategy.


In [190]:
vals = df.loc[df['margin'] != 0, 'margin'].values
q_1 = np.quantile(vals, 0.25)
q_2 = np.quantile(vals, 0.50)
q_3 = np.quantile(vals, 0.75)
q_3_iqr = np.quantile(vals, 0.75) + (q_3 - q_1)
#
figure = go.Figure()
figure.update_layout(go.Layout(margin = dict(l = 20, r = 20, t = 20, b = 20), template = 'simple_white', showlegend = False,
                               xaxis1 = {'showgrid': True, 'showline': True, 'mirror': True, 'titlefont': {'size': 20}, 'tickfont': {'size': 16},
                                         'tickangle': 0, 'title': 'Margin [$]'},
                               yaxis1 = {'showgrid': True, 'showline': True, 'mirror': True, 'titlefont': {'size': 20}, 'tickfont': {'size': 16},
                                         'tickformat': f'.{0}f'},
                               font = {'size': 28}, autosize = False, width = 900, height = 500, hovermode = 'closest'))
figure.add_trace(go.Histogram(x = vals, marker_color = 'lime'))
figure.add_vline(x = q_1, line_width = 2, line_dash = 'dash', line_color = 'green', annotation_text = '$Q_1$',
                 annotation_position = 'top right', annotation = {'font_color': 'green', 'font_size': 16, 'borderwidth': 10})
figure.add_vline(x = q_2, line_width = 2, line_dash = 'dash', line_color = 'yellow', annotation_text = '$Q_2$',
                 annotation_position = 'top right', annotation = {'font_color': 'yellow', 'font_size': 16, 'borderwidth': 10})
figure.add_vline(x = q_3, line_width = 2, line_dash = 'dash', line_color = 'red', annotation_text = '$Q_3$',
                 annotation_position = 'top right', annotation = {'font_color': 'red', 'font_size': 16, 'borderwidth': 10})
figure.add_vline(x = q_3_iqr, line_width = 2, line_dash = 'dash', line_color = 'cyan', annotation_text = '$Q_3 + \mathrm{IQR}$',
                 annotation_position = 'top right', annotation = {'font_color': 'cyan', 'font_size': 16, 'borderwidth': 10})

In [209]:
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day
df

,date,daily_profit,margin,cum_profit,max_equity,dd,year,month,day
0,2012-01-01,0.00,0.0,0.00,0.00,0.00,2012,1,1
1,2012-01-02,0.00,0.0,0.00,0.00,0.00,2012,1,2
2,2012-01-03,-1928.75,87000.0,-1928.75,0.00,-1928.75,2012,1,3
3,2012-01-04,-393.75,78000.0,-2322.50,0.00,-2322.50,2012,1,4
4,2012-01-05,1365.00,30000.0,-957.50,0.00,-957.50,2012,1,5
...,...,...,...,...,...,...,...,...,...
4654,2024-09-28,0.00,0.0,615603.75,642703.75,-27100.00,2024,9,28
4655,2024-09-29,16.25,30000.0,615620.00,642703.75,-27083.75,2024,9,29
4656,2024-09-30,-28.75,30000.0,615591.25,642703.75,-27112.50,2024,9,30
4657,2024-10-01,-1677.50,30000.0,613913.75,642703.75,-28790.00,2024,10,1
